In [5]:
# 라이브러리 준비
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os
from sklearn.model_selection import GroupKFold
import copy
import warnings
import torch.nn.functional as F

warnings.filterwarnings('ignore')

In [22]:
import importlib.metadata as metadata  # Python 3.8+

libs = ["pandas", "numpy", "torch", "sklearn", "os", "copy", "warnings"]

for name in libs:
    try:
        print(name, '==',metadata.version(name), sep='')
    except metadata.PackageNotFoundError:
        print(name, "not installed")


pandas==2.3.3
numpy==2.4.1
torch==2.10.0+cu128
sklearn not installed
os not installed
copy not installed
warnings not installed


In [6]:
# ==========================================
# 0. 설정 및 파라미터
# ==========================================
PITCH_X = 105.0
PITCH_Y = 68.

BATCH_SIZE = 64    # 샘플 수
EPOCHS = 35        # 반복 학습 횟수
PATIENCE = 7       # 검증 손실(Val Loss)이 7번(Epoch) 연속으로 좋아지지 않으면 학습 중단 (Early Stopping)
N_SPLITS = 5       # 교차 검증(Cross-Validation)의 그룹 수
MAX_SEQ_LEN = 50   # 과거 이벤트 최대 50개 까지 봄

# 모델 파라미터
HIDDEN_DIM = 128   # LSTM 내부 뉴런 개수 (클수록 모델의 표현력(기억 용량)이 좋아지지만, 과적합 위험이 커짐)
NUM_LAYERS = 2     # LSTM 층을 2겹으로 쌓음 => 복잡한 시계열 패턴을 학습할 수 있음
CONTEXT_LEN = 10   # 마지막 10개의 이벤트에 대해 Attention 메커니즘을 적용하여 가중치 계산
DROPOUT = 0.3      # 뉴런의 30%를 무작위로 끊어 과적합 방지

# 학습 파라미터
LR = 0.001         # 학습률(Learning Rate). Adam 옵티마이저 가중치를 이동시키는 보폭
ALPHA = 0.5        # Huber Loss + Euclidean Distance 결합 손실 함수용 파라미터
DELTA = 0.05       # - 이상치에 덜 민감하면서도, 실제 거리 오차를 줄이는 균형 잡힌 설정
LAMBDA_POS = 0.3   # 전체 손실(Total Loss) = 이동벡터 오차 + (Lamda_POS * 절대좌표 오차)
                   # - 이동 벡터를 맞추는 것이 목표이지만, 도착 위치(절대 좌표)도 맞추도록 학습 시킴
                   # - 적절히 모델의 공간 지각 능력을 돕도록 30% 정도의 비중만 둠

# 피처 리스트 (venue_id 추가됨)
FEAT_COLS = [
    'start_x_n', 'start_y_n',
    'type_id', 'result_id',
    'is_home', 'team_id_encoded',
    'distance', 'angle_norm',
    'velocity_norm', 'zone_normalized',
    'distance_bin_norm', 'relative_angle_norm',
    'prev_end_x_n', 'prev_end_y_n',
    'prev_dx_n', 'prev_dy_n',
    'prev_velocity_norm', 'prev_distance_bin_norm',
    'time_delta_n', 'prev_type_id',
    'venue_id',   
    'month_sin',  
    'month_cos'   
]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 디바이스: {device}")

np.random.seed(2025)
torch.manual_seed(2025)

사용 디바이스: cuda


In [7]:
# ==========================================
# 1. 데이터 로드 및 전처리 (Match Info & Augmentation)
# ==========================================

def load_match_info(path='match_info.csv'):
    if not os.path.exists(path):
        print(f"Warning: {path} not found. Skipping match info.")
        return None, {}
        
    df_match = pd.read_csv(path)
    
    # 1. Venue Encoding
    unique_venues = df_match['venue'].unique()
    venue2id = {v: i+1 for i, v in enumerate(unique_venues)}
    df_match['venue_id'] = df_match['venue'].map(venue2id).fillna(0).astype(int)
    
    # 2. 날짜 파싱 및 월(Month) 추출
    # 'game_date'를 datetime 객체로 변환
    df_match['game_date'] = pd.to_datetime(df_match['game_date'])
    
    # 월(Month) 추출 (1~12)
    df_match['month'] = df_match['game_date'].dt.month
    
    # 3. Cyclical Encoding (월 정보를 연속적인 원형 데이터로 변환)
    # 12월과 1월이 이어지도록 sin, cos 변환
    # month - 1을 하는 이유: 0~11로 맞춰주기 위함
    df_match['month_sin'] = np.sin(2 * np.pi * (df_match['month'] - 1) / 12)
    df_match['month_cos'] = np.cos(2 * np.pi * (df_match['month'] - 1) / 12)
    
    # 메타 정보 저장
    meta_match = {'venue2id': venue2id}
    
    # 필요한 컬럼 선택 (month_sin, month_cos 추가)
    # game_id를 기준으로 병합하므로 반드시 포함
    df_match_filtered = df_match[['game_id', 'venue_id', 'month_sin', 'month_cos']]
    
    return df_match_filtered, meta_match

def augment_data(df):
    print("데이터 증강 수행 중 (Flip Y)...")
    df_aug = df.copy()
    
    # 1. Y 좌표 반전
    df_aug['start_y'] = PITCH_Y - df_aug['start_y']
    df_aug['end_y'] = PITCH_Y - df_aug['end_y']
    
    # 2. 에피소드 ID 변경 (중복 방지)
    # game_id는 유지하여 GroupKFold에서 같은 폴드에 들어가게 함 (Data Leakage 방지)
    df_aug['game_episode'] = df_aug['game_episode'].astype(str) + '_aug'
    
    # 3. 합치기
    df_final = pd.concat([df, df_aug], axis=0).reset_index(drop=True)
    return df_final

def load_raw_and_process(train_path, match_path='match_info.csv', is_train=True):
    # 1. 메인 데이터 로드
    df = pd.read_csv(train_path)
    
    # 2. Match Info 병합
    df_match, match_meta = load_match_info(match_path)
    if df_match is not None:
        df = pd.merge(df, df_match, on='game_id', how='left')
        df['venue_id'] = df['venue_id'].fillna(0).astype(int)
    else:
        df['venue_id'] = 0
        match_meta = {}

    # 3. 정렬
    df = df.sort_values(['game_id', 'period_id', 'episode_id', 'time_seconds', 'action_id'])
    
    # 4. [Train Only] 데이터 증강
    if is_train:
        df = augment_data(df)
        
    df = df.reset_index(drop=True)
    return df, match_meta

In [8]:
# ==========================================
# 2. 피처 엔지니어링 (기존 로직 + Venue)
# ==========================================
def make_event_features(df: pd.DataFrame, meta: dict = None, is_train: bool = True) -> tuple:
    df_feat = df.copy()
    
    # 0. 카테고리 인코딩
    if is_train:
        type_cols = df_feat['type_name'].unique()
        type2id = {t: i+1 for i, t in enumerate(type_cols)}
        
        unique_results = df_feat['result_name'].fillna('Unknown').unique()
        if 'Unknown' not in unique_results:
            unique_results = np.append(unique_results, 'Unknown')
        res2id = {r: i+1 for i, r in enumerate(unique_results)}
        
        unique_teams = df_feat['team_id'].dropna().unique()
        team2id = {team: i+1 for i, team in enumerate(unique_teams)}
        
        # match_meta 병합
        if 'venue2id' in meta:
            venue2id = meta['venue2id']
        else:
             venue2id = {} # Fallback

        meta.update({
            'type2id': type2id,
            'res2id': res2id,
            'team2id': team2id,
        })
    else:
        type2id = meta['type2id']
        res2id = meta['res2id']
        team2id = meta['team2id']
    # 매핑 적용
    df_feat['type_id'] = df_feat['type_name'].map(type2id).fillna(0).astype(int)
    df_feat['result_id'] = df_feat['result_name'].fillna('Unknown').map(res2id).fillna(0).astype(int)
    df_feat['team_id_encoded'] = df_feat['team_id'].map(team2id).fillna(0).astype(int)
    
    # 1. 기본 이동 벡터 계산
    df_feat['dx'] = df_feat['end_x'] - df_feat['start_x']
    df_feat['dy'] = df_feat['end_y'] - df_feat['start_y']
    df_feat['distance'] = np.sqrt(df_feat['dx']**2 + df_feat['dy']**2)
    
    # 2. 좌표 정규화 (0~1)
    df_feat['start_x_n'] = df_feat['start_x'] / PITCH_X
    df_feat['start_y_n'] = df_feat['start_y'] / PITCH_Y
    df_feat['end_x_n'] = df_feat['end_x'] / PITCH_X
    df_feat['end_y_n'] = df_feat['end_y'] / PITCH_Y
    
    # 3. 패스 각도
    df_feat['angle_rad'] = np.arctan2(df_feat['dy'], df_feat['dx'])
    df_feat['angle_deg'] = np.degrees(df_feat['angle_rad'])
    df_feat['angle_norm'] = (df_feat['angle_deg'] + 180) / 360.0
    
    # 4. 시간 정보 (GroupBy 기반)
    g = df_feat.groupby('game_episode')
    df_feat['time_delta'] = g['time_seconds'].diff().fillna(0)
    df_feat['time_delta_n'] = np.clip(df_feat['time_delta'] / 7.0, 0, 1)
    
    # 5. 속도 계산
    df_feat['velocity'] = np.where(
        df_feat['time_delta'] > 0.1,
        df_feat['distance'] / df_feat['time_delta'],
        0
    )
    df_feat['velocity_norm'] = np.clip(df_feat['velocity'] / 17.0, 0, 1)
    
    # 6. 구역 피처 (3x3 그리드)
    df_feat['x_zone'] = np.clip((df_feat['start_x_n'] * 3).astype(int), 0, 2)
    df_feat['y_zone'] = np.clip((df_feat['start_y_n'] * 3).astype(int), 0, 2)
    df_feat['zone_id'] = df_feat['x_zone'] * 3 + df_feat['y_zone']
    df_feat['zone_normalized'] = df_feat['zone_id'] / 9.0
    
    # 7. 거리 구간
    def get_distance_bin(dist):
        if dist < 5:
            return 0
        elif dist < 15:
            return 1
        elif dist < 30:
            return 2
        else:
            return 3
    
    df_feat['distance_bin'] = df_feat['distance'].apply(get_distance_bin)
    df_feat['distance_bin_norm'] = df_feat['distance_bin'] / 3.0
    
    # 8. 상대 각도
    df_feat['prev_angle_deg'] = g['angle_deg'].shift(1).fillna(0)
    angle_diff = df_feat['angle_deg'] - df_feat['prev_angle_deg']
    angle_diff = np.where(angle_diff > 180, angle_diff - 360, angle_diff)
    angle_diff = np.where(angle_diff < -180, angle_diff + 360, angle_diff)
    df_feat['relative_angle_norm'] = (angle_diff + 180) / 360.0
    
    # 9. is_home 타입 변환
    df_feat['is_home'] = df_feat['is_home'].astype(int)

    # 10. Lag Features (직전 정보)
    df_feat['prev_end_x_n'] = g['end_x_n'].shift(1).fillna(0)
    df_feat['prev_end_y_n'] = g['end_y_n'].shift(1).fillna(0)
    df_feat['prev_dx_n'] = (g['dx'].shift(1) / PITCH_X).fillna(0)
    df_feat['prev_dy_n'] = (g['dy'].shift(1) / PITCH_Y).fillna(0)
    df_feat['prev_velocity_norm'] = g['velocity_norm'].shift(1).fillna(0)
    df_feat['prev_distance_bin_norm'] = g['distance_bin_norm'].shift(1).fillna(0)
    df_feat['prev_type_id'] = g['type_id'].shift(1).fillna(0)
    
    # 11. NaN/Inf 처리
    df_feat = df_feat.fillna(0)
    df_feat = df_feat.replace([np.inf, -np.inf], 0)
    
    return df_feat, meta

In [9]:
# ==========================================
# 3. 시퀀스 구축 (Target에 절대좌표 추가)
# ==========================================

def build_episode_sequences(df, meta, max_len=50, min_pass_in_episode=1, feat_cols=FEAT_COLS):
    X_list, y_list, info_list = [], [], []
    
    for ep, g in df.groupby('game_episode'):    # 에피소드 그룹화
        pass_rows = g[g['type_name'] == 'Pass']
        if len(pass_rows) < min_pass_in_episode: continue
        
        last_pass_idx = pass_rows.index[-1]
        last_pass = df.loc[last_pass_idx]

        # 절대 좌표와 변화량 모두 계산
        start_x_n, start_y_n = last_pass['start_x_n'], last_pass['start_y_n']
        end_x_n, end_y_n = last_pass['end_x_n'], last_pass['end_y_n'] # 절대좌표 타겟
        target_dx, target_dy = end_x_n - start_x_n, end_y_n - start_y_n # 변화량

        # 입력 시퀀스 구축
        g_input = g[g.index < last_pass_idx].copy()
        if len(g_input) == 0: continue
        
        seq = g_input[feat_cols].copy()
        seq_vals = seq.to_numpy(dtype=float)
        
        seq_len = len(seq_vals)

        # 길이 고정
        if seq_len > max_len:
            seq_vals = seq_vals[-max_len:]
        elif seq_len < max_len:
            pad = np.zeros((max_len - seq_len, seq_vals.shape[1]), dtype=float) # Zero Padding: 부족한 만큼 0으로 채움
            seq_vals = np.concatenate([pad, seq_vals], axis=0)                  # - 앞에 넣는 이유: LSTM 시퀀스의 최근 정보를 더 잘 기억하기 때문에
        
        X_list.append(seq_vals)
        # Target: [dx, dy, end_x, end_y] -> Multi-task learning
        y_list.append([target_dx, target_dy, end_x_n, end_y_n]) 

        # 메타 정보 저장
        info_list.append({
            'game_episode': ep,
            'game_id': g['game_id'].iloc[0], # game_id는 원본 유지 (split용)
            'last_start_x_n': start_x_n,
            'last_start_y_n': start_y_n,
        })
    
    X = np.stack(X_list, axis=0) if X_list else np.array([])
    y = np.array(y_list, dtype=float) if y_list else np.array([])
    info_df = pd.DataFrame(info_list)
    return X, y, info_df, feat_cols

In [18]:
# ==========================================
# 3.5. 데이터셋 클래스 정의
# ==========================================

class PassTraceDataset(Dataset):
    # 1. 초기화
    def __init__(self, X, y=None):
        # NumPy 배열을 PyTorch의 실수형 텐서(FloatTensor)로 변환
        self.X = torch.FloatTensor(X)
        # y(정답)가 있으면 변환하고, 없으면(테스트용) None으로 둠
        self.y = torch.FloatTensor(y) if y is not None else None
    
    # 2. 길이 반환
    def __len__(self):
        return len(self.X)
    
    # 3. 데이터 꺼내기
    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]    # 학습용
        return self.X[idx]    # 테스트용

In [ ]:
# ==========================================
# 4. 모델 정의(Attention + Multi-task)
# ==========================================

class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_dim * 2, 1) # Bi-LSTM(양방향)이라 정보량이 *2배
    
    def forward(self, lstm_output):
        # 1. 점수 계산 (Scoring)
        # - 각 시점(t)마다 점수를 계산
        scores = self.attn(lstm_output)

        # 2. 가중치 변환 (Softmax)
        # - 점수들을 확률처럼 변환(전체 합 1)
        attn_weights = F.softmax(scores, dim=1) 

        # 3. 컨텍스트 벡터 생성 (Weighted Sum)
        # - 각 시점의 LSTM 출력값에 위에서 구한 가중치를 곱해서 더함
        # - 즉, 중요한 시점의 정보는 많이 반영, 안 중요한 정보는 무시
        context = torch.sum(attn_weights * lstm_output, dim=1) 
        return context, attn_weights
        
class SoccerLSTM_Advanced(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers=2, dropout=0.3, context_len=10):
        super(SoccerLSTM_Advanced, self).__init__()

        # 1. Bi-LSTM
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,    # 과거, 미래 쌍방향 읽음 => 출력크기 hidden_dim * 2
            dropout=dropout if num_layers > 1 else 0
        )

        # 2. 집중 범위
        self.context_len = context_len    # 최근 10개 이벤트에 집중
        self.attention = Attention(hidden_dim)
        
        # 3. 멀티태스크 헤드
        # Head 1: 이동 벡터 예측 (dx, dy)
        self.fc_delta = nn.Sequential(
            nn.Linear(hidden_dim * 2, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 2)
        )
        
        # Head 2: 절대 좌표 예측 (x, y) - Auxiliary Task
        self.fc_pos = nn.Sequential(
            nn.Linear(hidden_dim * 2, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 2),
            nn.Sigmoid() # 좌표는 0~1 사이 정규화되어 있음
        )
    
    # 순전파
    def forward(self, x):
        # 1. 흐름 읽기 (LSTM)
        # lstm_out: 전체 시퀀스의 특징을 추출한 데이터
        lstm_out, _ = self.lstm(x)

        # 2. 최근 상황 잘라내기
        # - 전체 50개 중 마지막 10(context_len)개만 잘라냄
        ctx_seq = lstm_out[:, -self.context_len:, :]

        # 3. 중요도 판단(Attention)
        # 10개 중에서 특히 중요한 순간을 골라 요약 정보(context)로 만듦
        context, _ = self.attention(ctx_seq)

        # 4. 두 가지 예측 수행 (Multi-task)
        # - 요약 정보(context)를 바탕으로 두 가지 질문에 답함
        pred_delta = self.fc_delta(context) # Q1. 이동 벡터는? (dx, dy)
        pred_pos = self.fc_pos(context)     # Q2. 절대 위치는? (x, y)
        
        return pred_delta, pred_pos

In [11]:
# ==========================================
# 5. 손실 함수 (Combined Multi-task Loss)
# ==========================================
class CombinedMultiTaskLoss(nn.Module):
    # 1. 채점 도구 준비
    def __init__(self, delta=0.05, alpha=0.5, lambda_pos=0.3):
        super(CombinedMultiTaskLoss, self).__init__()
        self.delta = delta            # Huber Loss의 기준점 (작은 오차 vs 큰 오차 구분선)
        self.alpha = alpha            # Huber Loss와 Euclidean Distance 사이의 비중
        self.lambda_pos = lambda_pos  # 메인 문제(이동량)와 보조 문제(위치) 사이의 비중
        self.mse = nn.MSELoss()       # 일반적인 제곱 오차 계산기

    # 2. 예측 패스의 방향과 거리(delta)가 얼마나 틀렸는지 계산
    def huber_euclid(self, pred, target):
        # 2.1. Huber Loss (휴버 손실)
        # - 이상치에 휘둘리지 않게, 작은 오차에는 정교함을 높이고, 큰 오차에는 적당히 넘어감
        diff = pred - target
        abs_diff = torch.abs(diff)
        huber = torch.where(
            abs_diff < self.delta,        
            0.5 * diff**2 / self.delta,   # 오차가 아주 작으면 (0.05 미만) 제곱해서 부드럽게 (MSE 처럼)
            abs_diff - 0.5 * self.delta   # 오차가 크면 절대값으로 (MAE 처럼)
        ).sum(dim=1)
        # 2.2. Euclidean Distance
        # - 실제 물리적 거리를 줄임
        euclid = torch.sqrt(torch.sum(diff**2, dim=1) + 1e-8)     # 0으로 나누는 오류 방지
        # 2.3. 결합 (alpha)
        # - Huber, Euclidean 50:50 으로 섞어서 점수를 냄
        return self.alpha * huber.mean() + (1 - self.alpha) * euclid.mean()

    # 3. 전체 점수 (Multi-task)
    def forward(self, pred_delta, target_delta, pred_pos, target_pos):
        # 1. Delta Loss (Main): 패스 변화량(dx, dy) 채점
        loss_delta = self.huber_euclid(pred_delta, target_delta)
        
        # 2. Position Loss (Auxiliary) - MSE 사용: 절대 위치(x, y) 채점
        loss_pos = self.mse(pred_pos, target_pos)
        
        # 3. Combined: 최종 점수 합산
        return loss_delta + self.lambda_pos * loss_pos

In [13]:
# ==========================================
# 6. 학습 및 실행 로직
# ==========================================

def train_advanced_model(X, y, game_ids, n_splits=5, epochs=30, batch_size=64):
    input_dim = X.shape[2]
    gkf = GroupKFold(n_splits=n_splits)
    trained_models = []
    
    # game_ids에서 _aug 제거 (GroupKFold용)
    # augmentation 시 game_id는 유지했으므로 그대로 사용하면 됨
    # 1. GroupKFold 교차 검증 준비
    for fold, (train_idx, val_idx) in enumerate(gkf.split(X, y, game_ids)):
        print(f"Fold {fold+1}/{n_splits}")
        
        X_train, y_train = X[train_idx], y[train_idx]
        X_val, y_val = X[val_idx], y[val_idx]
        
        # 2. 데이터 로더 설정
        train_ds = PassTraceDataset(X_train, y_train)
        val_ds = PassTraceDataset(X_val, y_val)
        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True) # batch_size(64개)씩/ 학습 데이터 순서 무작위
        val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)    # 검증 데이터 순서 상관 X 

        # 3. 모델 및 학습 도구 초기화
        model = SoccerLSTM_Advanced(input_dim, HIDDEN_DIM, NUM_LAYERS, DROPOUT, CONTEXT_LEN).to(device)
        criterion = CombinedMultiTaskLoss(DELTA, ALPHA, LAMBDA_POS)   # 손실함수 준비
        optimizer = optim.Adam(model.parameters(), lr=LR)             # 옵티마이저 모듈/ 가중치 0.001
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3)   # 학습 정체 시 학습률 낮춰 세밀하게 학습
        
        best_loss = float('inf')
        best_state = None
        patience_cnt = 0
        
        for epoch in range(epochs):
            # 4. 학습 루프 *중요*
            model.train() # 학습 모드 전환 (Dropout 켜짐)
            train_loss = 0
            for bx, by in train_loader:
                bx, by = bx.to(device), by.to(device)
                # 정답 데이터(y)
                target_delta = by[:, :2] # dx, dy (이동 벡터)
                target_pos = by[:, 2:]   # x, y (절대 위치)
                
                optimizer.zero_grad()
                pred_delta, pred_pos = model(bx)  # 모델 예측

                # 멀티태스크 손실 계산
                loss = criterion(pred_delta, target_delta, pred_pos, target_pos)
                loss.backward()     # 기울기 계산
                optimizer.step()    # 가중치 업데이트
                train_loss += loss.item()
                
            # 5.검증 루프 Validation
            model.eval()    # 평가 모드 전환 (Dropout 꺼짐)
            val_loss = 0
            with torch.no_grad():  # 기울기 계산 끔 (메모리 절약)
                for bx, by in val_loader:
                    bx, by = bx.to(device), by.to(device)
                    target_delta = by[:, :2]
                    target_pos = by[:, 2:]
                    
                    p_delta, p_pos = model(bx)
                    loss = criterion(p_delta, target_delta, p_pos, target_pos)
                    val_loss += loss.item()
            
            avg_val_loss = val_loss / len(val_loader)
            scheduler.step(avg_val_loss)

            # 6. 조기 종료 (Earlry Stopping) 및 모델 저장
            if avg_val_loss < best_loss:
                best_loss = avg_val_loss
                best_state = copy.deepcopy(model.state_dict())  # 최고 점수 시점 저장
                patience_cnt = 0
                print(f"  Ep {epoch+1}: Val Loss {avg_val_loss:.4f} (Best)")
            else:
                patience_cnt += 1
                if patience_cnt >= PATIENCE:
                    print("Early Stopping")
                    break
        
        model.load_state_dict(best_state)
        trained_models.append(model)
        
    return trained_models

In [14]:
# ==========================================
# 7. 테스트 데이터 준비 (Test Data Preparation)
# ==========================================

def prepare_test_data_advanced(test_csv_path, match_csv_path, meta, feat_cols, max_len=50):
    print(f"테스트 데이터 인덱스 로드 중: {test_csv_path}")
    df_test_index = pd.read_csv(test_csv_path) # 인덱스 파일 로드
    
    # 1. Match Info 병합 (Venue, Month 정보 등) - 인덱스 데이터프레임에 병합
    if os.path.exists(match_csv_path):
        # load_match_info 함수를 재사용하거나 직접 로드
        # 여기서는 로직 명확성을 위해 직접 로드 및 처리
        df_match = pd.read_csv(match_csv_path)
        
        # Meta 정보 활용 (Venue ID)
        venue2id = meta.get('venue2id', {})
        df_match['venue_id'] = df_match['venue'].map(venue2id).fillna(0).astype(int)
        
        # 날짜 파싱 및 Cyclical Encoding (Train과 동일하게)
        if 'game_date' in df_match.columns:
            df_match['game_date'] = pd.to_datetime(df_match['game_date'])
            df_match['month'] = df_match['game_date'].dt.month
            df_match['month_sin'] = np.sin(2 * np.pi * (df_match['month'] - 1) / 12)
            df_match['month_cos'] = np.cos(2 * np.pi * (df_match['month'] - 1) / 12)
        
        # 필요한 컬럼 준비 (venue_id, month_sin, month_cos가 있는지 확인)
        cols_to_merge = ['game_id', 'venue_id']
        if 'month_sin' in df_match.columns:
            cols_to_merge.extend(['month_sin', 'month_cos'])
            
        # 병합
        df_test_index = pd.merge(df_test_index, df_match[cols_to_merge], on='game_id', how='left')
        
        # 결측치 채우기
        for col in ['venue_id', 'month_sin', 'month_cos']:
            if col in df_test_index.columns:
                df_test_index[col] = df_test_index[col].fillna(0)
                if col == 'venue_id':
                    df_test_index[col] = df_test_index[col].astype(int)
    else:
        print("Warning: match_info.csv not found for test. Filling meta features with 0.")
        df_test_index['venue_id'] = 0
        df_test_index['month_sin'] = 0
        df_test_index['month_cos'] = 0

    # 3. 데이터 컨테이너
    X_list = []
    episode_ids = []
    last_start_x_n_list = []
    last_start_y_n_list = []

    print(f"테스트 데이터 개별 파일 로드 및 처리 중... (총 {len(df_test_index)}개)")
    
    # test.csv의 각 행(에피소드)을 순회
    for idx, row in df_test_index.iterrows():
        episode_id = row['game_episode']
        file_path = row['path']
        
        # [중요] 개별 에피소드 CSV 파일 로드
        try:
            df_ep = pd.read_csv(file_path)
        except FileNotFoundError:
            print(f"File not found: {file_path}")
            continue

        # [중요] 개별 파일 내부 정렬
        df_ep = df_ep.sort_values(['time_seconds', 'action_id']).reset_index(drop=True)
        
        # Match Info(Venue, Month) 정보를 에피소드 데이터에 붙여넣기
        if 'venue_id' in row:
            df_ep['venue_id'] = row['venue_id']
        if 'month_sin' in row:
            df_ep['month_sin'] = row['month_sin']
            df_ep['month_cos'] = row['month_cos']
        
        # 피처 엔지니어링 (is_train=False)
        df_ep_feat, _ = make_event_features(df_ep, meta=meta, is_train=False)
        
        # 시퀀스 구축
        # 마지막 패스 찾기
        pass_rows = df_ep_feat[df_ep_feat['type_name'] == 'Pass']
        if len(pass_rows) == 0:
            continue
            
        last_pass_idx = pass_rows.index[-1]
        last_pass = df_ep_feat.loc[last_pass_idx]
        
        last_x = last_pass['start_x_n']
        last_y = last_pass['start_y_n']
        
        # 입력 시퀀스: 마지막 패스 '이전' 까지
        df_input = df_ep_feat[df_ep_feat.index < last_pass_idx].copy()
        
        if len(df_input) == 0:
            continue
            
        # Feature Selection & Padding
        # feat_cols에 없는 컬럼이 있으면 0으로 채움 (안전장치)
        for col in feat_cols:
            if col not in df_input.columns:
                df_input[col] = 0
                
        seq = df_input[feat_cols].fillna(0).replace([np.inf, -np.inf], 0)
        seq_vals = seq.to_numpy(dtype=float)
        
        seq_len = len(seq_vals)
        if seq_len > max_len:
            seq_vals = seq_vals[-max_len:]
        elif seq_len < max_len:
            pad = np.zeros((max_len - seq_len, seq_vals.shape[1]), dtype=float)
            seq_vals = np.concatenate([pad, seq_vals], axis=0)
            
        X_list.append(seq_vals)
        episode_ids.append(episode_id)
        last_start_x_n_list.append(last_x)
        last_start_y_n_list.append(last_y)

    X_test = np.stack(X_list, axis=0) if X_list else np.array([])
    last_start_x_n_arr = np.array(last_start_x_n_list)
    last_start_y_n_arr = np.array(last_start_y_n_list)
    
    return X_test, episode_ids, last_start_x_n_arr, last_start_y_n_arr

In [20]:
# ==========================================
# 8. 예측 및 제출 (Prediction & Submission)
# ==========================================

def predict_and_submit_advanced(trained_models, X_test, episode_ids, last_x_arr, last_y_arr, submission_path='submission.csv'):
    # Test Dataset & Loader
    test_ds = PassTraceDataset(X_test) # y=None
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)
    
    print(f"\n모델 예측 수행 중 (Ensemble Size: {len(trained_models)})...")
    
    all_fold_deltas = []
    
    # Fold별 추론
    for i, model in enumerate(trained_models):
        model.eval()
        fold_deltas = []
        
        with torch.no_grad():
            for bx in test_loader:
                bx = bx.to(device)
                
                # 모델 출력: (delta, pos)
                pred_delta, pred_pos = model(bx)
                
                # CPU로 이동
                fold_deltas.append(pred_delta.cpu().numpy())
                
        all_fold_deltas.append(np.concatenate(fold_deltas, axis=0))
    
    # 앙상블 (평균)
    avg_deltas = np.mean(all_fold_deltas, axis=0) # (N, 2) -> dx_n, dy_n
    
    # 좌표 복원 (Denormalization)
    pred_dx_n = avg_deltas[:, 0]
    pred_dy_n = avg_deltas[:, 1]
    
    # 최종 좌표 계산: 시작점 + 변화량
    pred_end_x_n = last_x_arr + pred_dx_n
    pred_end_y_n = last_y_arr + pred_dy_n
    
    # 실제 스케일(m)로 변환
    pred_end_x = pred_end_x_n * PITCH_X
    pred_end_y = pred_end_y_n * PITCH_Y
    
    # 경기장 범위 밖으로 나가는 값 Clip (Safety Guard)
    pred_end_x = np.clip(pred_end_x, 0, PITCH_X)
    pred_end_y = np.clip(pred_end_y, 0, PITCH_Y)
    
    # 데이터프레임 생성
    submission = pd.DataFrame({
        'game_episode': episode_ids,
        'end_x': pred_end_x,
        'end_y': pred_end_y
    })
    
    # 저장
    submission.to_csv(submission_path, index=False)
    print(f"\n✓ 예측 완료 및 저장됨: {submission_path}")
    print(submission.head())
    
    return submission

In [21]:
# ==========================================
# 실행 (Main)
# ==========================================
if __name__ == "__main__":
    # 1. 데이터 로드 및 전처리
    print("1. 데이터 로드 및 전처리...")
    df_train, match_meta = load_raw_and_process('train.csv', 'match_info.csv', is_train=True)
    
    # 2. 피처 엔지니어링
    print("2. 피처 엔지니어링...")
    df_train_feat, meta = make_event_features(df_train, meta=match_meta, is_train=True)
    
    # 3. 시퀀스 구축
    print("3. 시퀀스 구축...")
    X, y, info_df, feat_cols = build_episode_sequences(df_train_feat, meta)
    
    print(f"학습 데이터 크기: {X.shape}") 
    
    # 4. 학습 및 실행 로직
    print("Step 4: Training...")
    models = train_advanced_model(X, y, info_df['game_id'].values)

    # 5. 테스트 데이터 준비
    print("\n[Step 5] Predicting on Test Data...")
    
    X_test, test_ids, last_x_test, last_y_test = prepare_test_data_advanced(
        test_csv_path='test.csv',
        match_csv_path='match_info.csv',
        meta=meta,           # 학습 시 생성된 메타데이터
        feat_cols=feat_cols, # venue_id가 포함된 피처 리스트
        max_len=MAX_SEQ_LEN
    )
    
    print(f"Test Set Shape: {X_test.shape}")
    
    # 6. 예측 및 제출
    if len(models) > 0:
        submission = predict_and_submit_advanced(
            trained_models=models,
            X_test=X_test,
            episode_ids=test_ids,
            last_x_arr=last_x_test,
            last_y_arr=last_y_test,
            submission_path='submission_advanced.csv'
        )
    else:
        print("Error: 학습된 모델이 없습니다.")

Step 1: Data Loading & Augmentation...
데이터 증강 수행 중 (Flip Y)...
Step 2: Feature Engineering...
Step 3: Sequence Building...
학습 데이터 크기: (30856, 50, 23)
Step 4: Training...
Fold 1/5
  Ep 1: Val Loss 0.2422 (Best)
  Ep 2: Val Loss 0.2276 (Best)
  Ep 3: Val Loss 0.2232 (Best)
  Ep 4: Val Loss 0.2185 (Best)
  Ep 5: Val Loss 0.2147 (Best)
  Ep 6: Val Loss 0.2044 (Best)
  Ep 7: Val Loss 0.1986 (Best)
  Ep 9: Val Loss 0.1963 (Best)
  Ep 10: Val Loss 0.1934 (Best)
  Ep 11: Val Loss 0.1918 (Best)
  Ep 12: Val Loss 0.1912 (Best)
  Ep 13: Val Loss 0.1894 (Best)
  Ep 18: Val Loss 0.1880 (Best)
Early Stopping
Fold 2/5
  Ep 1: Val Loss 0.2406 (Best)
  Ep 2: Val Loss 0.2263 (Best)
  Ep 3: Val Loss 0.2216 (Best)
  Ep 4: Val Loss 0.2193 (Best)
  Ep 5: Val Loss 0.2128 (Best)
  Ep 6: Val Loss 0.2048 (Best)
  Ep 7: Val Loss 0.2003 (Best)
  Ep 8: Val Loss 0.1960 (Best)
  Ep 9: Val Loss 0.1960 (Best)
  Ep 10: Val Loss 0.1912 (Best)
  Ep 11: Val Loss 0.1893 (Best)
  Ep 12: Val Loss 0.1876 (Best)
  Ep 14: Val L